# Manutenzione preventiva - 

# Manutenzione preventiva - DoS Attacks

In [1]:
import pandas as pd
import json as json
import ipaddress as ip 
import numpy as np

In [2]:
clean_cap_30m = open('.\\captures\\captures1_v2\\clean\\eth2dump-clean-0,5h_1.json')
data_cc_30m = json.load(clean_cap_30m)
clean_cap_30m.close()

In [3]:
interesting_layers = ["eth", "arp", "icmp", "ip", "tcp", "udp", "mbtcp"]
timestamp_field_name = 'frame.time_epoch'
ipat_field_name = 'inter.packet_arrival_time'
eth_fields = ['eth.dst', 'eth.src', 'eth.type', 'eth.len']
arp_fields = ['arp.hw_type', 'arp.proto_type', 'arp.hw_size', 'arp.proto_size']
icmp_fields = ['icmp.type', 'icmp.code', 'icmp.checksum', 'icmp.checksum_status']
ip_fields = ['ip.version', 'ip.hdr_len', 'ip.dsfield', 'ip.dsfield_dscp', 'ip.dsfield_ecn',
             'ip.len', 'ip.id', 'ip.flags', 'ip.flags_rb', 'ip.flags_df', 'ip.flags_mf',
             'ip.frag_offset', 'ip.ttl', 'ip.proto', 'ip.checksum', 'ip.checksum_status', 'ip.src', 'ip.dst']
tcp_fields = ['tcp.srcport', 'tcp.dstport', 'tcp.stream', 'tcp.len', 'tcp.seq', 'tcp.nxtseq', 'tcp.ack',
              'tcp.hdr_len', 'tcp.flags', 'tcp.flags_res', 'tcp.flags_ae', 'tcp.flags_cwr', 'tcp.flags_ece',
              'tcp.flags_urg', 'tcp.flags_ack', 'tcp.flags_push', 'tcp.flags_reset', 'tcp.flags_syn', 'tcp.flags_fin',
              'tcp.flags_str', 'tcp.window_size_value', 'tcp.window_size', 'tcp.window_size_scalefactor', 'tcp.checksum',
              'tcp.checksum_status', 'tcp.urgent_pointer']
udp_fields = ['udp.srcport', 'udp.dstport', 'udp.port', 'udp.length', 'udp.checksum', 'udp.checksum_status']
mbtcp_fields = ['mbtcp.trans_id', 'mbtcp.prot_id', 'mbtcp.len', 'mbtcp.unit_id']
interesting_layer_fields = [eth_fields, arp_fields, icmp_fields, ip_fields, tcp_fields, udp_fields, mbtcp_fields]
df_columns = [timestamp_field_name] + eth_fields + arp_fields + icmp_fields + ip_fields + tcp_fields + udp_fields + mbtcp_fields
packets_values = []

In [4]:
counter = 0
for packet in data_cc_30m:
    tmp_pkt = []

    tmp_pkt.append(packet['_source']['layers']['frame']['frame.time_epoch'])
    
    for i in range(len(interesting_layers)):
        if interesting_layers[i] in packet['_source']['layers']:
            for field in interesting_layer_fields[i]:
                if field in packet['_source']['layers'][interesting_layers[i]]:
                    tmp_pkt.append(packet['_source']['layers'][interesting_layers[i]][field])
                else:
                    tmp_pkt.append(np.nan)
        else:
            for field in interesting_layer_fields[i]:
                tmp_pkt.append(np.nan)
    packets_values.append(tmp_pkt)

print(len(packets_values[1]))
print(len(packets_values))
      

67
35430


In [5]:
# Costruzione dataframe
df_clean_30m = pd.DataFrame(data=packets_values, columns=df_columns)
# Conversione colonna timestamp a float
df_clean_30m['frame.time_epoch'] = df_clean_30m['frame.time_epoch'].astype('float')
# Inserimento colonna inter packet arrival time
df_clean_30m[ipat_field_name] = df_clean_30m['frame.time_epoch'].diff()

df_clean_30m.columns

Index(['frame.time_epoch', 'eth.dst', 'eth.src', 'eth.type', 'eth.len',
       'arp.hw_type', 'arp.proto_type', 'arp.hw_size', 'arp.proto_size',
       'icmp.type', 'icmp.code', 'icmp.checksum', 'icmp.checksum_status',
       'ip.version', 'ip.hdr_len', 'ip.dsfield', 'ip.dsfield_dscp',
       'ip.dsfield_ecn', 'ip.len', 'ip.id', 'ip.flags', 'ip.flags_rb',
       'ip.flags_df', 'ip.flags_mf', 'ip.frag_offset', 'ip.ttl', 'ip.proto',
       'ip.checksum', 'ip.checksum_status', 'ip.src', 'ip.dst', 'tcp.srcport',
       'tcp.dstport', 'tcp.stream', 'tcp.len', 'tcp.seq', 'tcp.nxtseq',
       'tcp.ack', 'tcp.hdr_len', 'tcp.flags', 'tcp.flags_res', 'tcp.flags_ae',
       'tcp.flags_cwr', 'tcp.flags_ece', 'tcp.flags_urg', 'tcp.flags_ack',
       'tcp.flags_push', 'tcp.flags_reset', 'tcp.flags_syn', 'tcp.flags_fin',
       'tcp.flags_str', 'tcp.window_size_value', 'tcp.window_size',
       'tcp.window_size_scalefactor', 'tcp.checksum', 'tcp.checksum_status',
       'tcp.urgent_pointer', 'udp.src

In [32]:
df_clean_30m.filter(like='udp', axis=1)

,udp.srcport,udp.dstport,udp.port,udp.length,udp.checksum,udp.checksum_status
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
35425,NaN,NaN,NaN,NaN,NaN,NaN
35426,NaN,NaN,NaN,NaN,NaN,NaN
35427,NaN,NaN,NaN,NaN,NaN,NaN
35428,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# Conversione del MAC Address da esadecimale a decimale
# flags_ tutti nulli
# ip.checksum_status tutti nulli
# arp tutti null
# icmp tutti nulli (da verificare)
# tcp.checksum_status tutti null
# eliminare le colonne che non servono piu alla fine e inserire i campi binari e vedere il numero di feature
df_clean_30m['eth.dst_cleaned'] = df_clean_30m['eth.dst'].str.replace(":", "", regex=False)
df_clean_30m['eth.dst_int'] = df_clean_30m['eth.dst_cleaned'].apply(lambda x : int(x, 16))
df_clean_30m['eth.src_cleaned'] = df_clean_30m['eth.src'].str.replace(":", "", regex=False)
df_clean_30m['eth.src_int'] = df_clean_30m['eth.src_cleaned'].apply(lambda x : int(x, 16))
df_clean_30m['eth.type'] = df_clean_30m['eth.type'].apply(lambda x : int(x, 16))
df_clean_30m['eth.len'] = df_clean_30m['eth.len'].astype(float)
df_clean_30m.drop(['eth.dst_cleaned', 'eth.dst', 'eth.src_cleaned', 'eth.src'], axis=1)

df_clean_30m['ip.version'] = df_clean_30m['ip.version'].astype(float)
df_clean_30m['ip.hdr_len'] = df_clean_30m['ip.hdr_len'].astype(float)
df_clean_30m['ip.src'].replace({np.nan: '169.254.0.0'}, inplace=True)
df_clean_30m['ip.dst'].replace({np.nan: '169.254.0.0'}, inplace=True)
df_clean_30m['ip.src'] = df_clean_30m['ip.src'].apply(lambda x : int(ip.IPv4Address(x)))
df_clean_30m['ip.dst'] = df_clean_30m['ip.dst'].apply(lambda x : int(ip.IPv4Address(x)))
df_clean_30m['ip.dsfield'] = df_clean_30m['ip.dsfield'].apply(lambda x : int(x, 16))
df_clean_30m['ip.dsfield_dscp'] = df_clean_30m['ip.dsfield_dscp'].astype(float)
df_clean_30m['ip.dsfield_ecn'] = df_clean_30m['ip.dsfield_ecn'].astype(float)
df_clean_30m['ip.len'] = df_clean_30m['ip.len'].astype(int)
df_clean_30m['ip.id'] = df_clean_30m['ip.id'].apply(lambda x : int(x, 16))
df_clean_30m['ip.flags'] = df_clean_30m['ip.flags'].apply(lambda x : int(x, 16))
df_clean_30m['ip.frag_offset'] = df_clean_30m['ip.frag_offset'].astype(float)
df_clean_30m['ip.ttl'] = df_clean_30m['ip.ttl'].astype(int)
df_clean_30m['ip.proto'] = df_clean_30m['ip.proto'].astype(int)
df_clean_30m['ip.checksum'] = df_clean_30m['ip.checksum'].apply(lambda x : int(x, 16))




df_clean_30m.loc[df_clean_30m['tcp.srcport'] == '0']
df_clean_30m['tcp.srcport'].replace({np.nan: '0'}, inplace=True)
df_clean_30m['tcp.dstport'].replace({np.nan: '0'}, inplace=True)
df_clean_30m['tcp.srcport'] = df_clean_30m['tcp.srcport'].apply(lambda x : int(x))
df_clean_30m['tcp.dstport'] = df_clean_30m['tcp.dstport'].apply(lambda x : int(x))
df_clean_30m['tcp.stream'] = df_clean_30m['tcp.stream'].astype(float)
df_clean_30m['tcp.len'] = df_clean_30m['tcp.len'].astype(float)
df_clean_30m['tcp.seq'] = df_clean_30m['tcp.seq'].astype(float)
df_clean_30m['tcp.nxtseq'] = df_clean_30m['tcp.nxtseq'].astype(float)
df_clean_30m['tcp.ack'] = df_clean_30m['tcp.ack'].astype(float)
df_clean_30m['tcp.hdr_len'] = df_clean_30m['tcp.hdr_len'].astype(float)
df_clean_30m['tcp.window_size_value'] = df_clean_30m['tcp.window_size_value'].astype(float)
df_clean_30m['tcp.window_size'] = df_clean_30m['tcp.window_size'].astype(float)
df_clean_30m['tcp.window_size_scalefactor'] = df_clean_30m['tcp.window_size_scalefactor'].astype(float)
df_clean_30m['tcp.urgent_pointer'] = df_clean_30m['tcp.urgent_pointer'].astype(float)
df_clean_30m['tcp.flags'] = df_clean_30m['tcp.flags'].apply(lambda x : int(x, 16))
df_clean_30m['tcp.checksum'] = df_clean_30m['tcp.checksum'].apply(lambda x : int(x, 16))


df_clean_30m['mbtcp.trans_id'] = df_clean_30m['mbtcp.trans_id'].astype(float)
df_clean_30m['mbtcp.prot_id'] = df_clean_30m['mbtcp.prot_id'].astype(float)
df_clean_30m['mbtcp.len'] = df_clean_30m['mbtcp.len'].astype(float)
df_clean_30m['mbtcp.unit_id'] = df_clean_30m['mbtcp.unit_id'].astype(float)







0        NaN
1         39
2        NaN
3        NaN
4        NaN
        ... 
35425    NaN
35426    NaN
35427    NaN
35428    NaN
35429    NaN
Name: eth.len, Length: 35430, dtype: object